# Data Exploration - Lyft Data Challenge

By: Adam Li
Date: 09/07/19

Background (copied):

data/driver_ids.csv

    * driver_id Unique identifier for a driver
    * driver_onboard_date Date on which driver was on-boarded

data/ride_ids.csv

    * driver_id Unique identifier for a driver
    * ride_id Unique identifier for a ride that was completed by the driver
    * ride_distance Ride distance in meters
    * ride_duration Ride duration in seconds
    * ride_prime_time Prime Time applied on the ride

data/ride_timestamps.csv
    
    * ride_id Unique identifier for a ride
    * event event describes the type of event (see below)
    * timestamp Time of event

You can assume that:
* All rides in the data set occurred in San Francisco
* All timestamps in the data set are in UTC

After exploring and analyzing the data, please:

1. Recommend a Driver's Lifetime Value (i.e., the value of a driver to Lyft over the entire
projected lifetime of a driver).

2. Please answer the following questions:

    * What are the main factors that affect a driver's lifetime value? 
    * What is the average projected lifetime of a driver? That is, once a driver is
    onboarded, how long do they typically continue driving with Lyft? 
    * Do all drivers act alike? Are there specific segments of drivers that generate more
    value for Lyft than the average driver?
    * What actionable recommendations are there for the business?
    
3. Prepare and submit a writeup of your findings for consumption by a cross-functional audience.

Here is an overview of the event types:
1. requested_at passenger requested a ride
2. accepted_at driver accepted a passenger request
3. arrived_at driver arrived at pickup point
4. picked_up_at driver picked up the passenger
5. dropped_off_at driver dropped off a passenger at destination

You can make the following assumptions about the Lyft rate card:

    * Base Fare 2.00
    * Cost per Mile 1.15
    * Cost per Minute 0.22
    * Service Fee 1.75
    * Minimum Fare 5.00
    * Maximum Fare 400.00

### Summary:
It's a pretty standard dataset, where the goal is a regression problem in the form of estimating a driver's lifetime value (DLV). In addition, we need to come up with answers to: i) the main factors that affect DLV, ii) average DLV (how long do they stay w/ lyft), and iii) driver's behavior distribution (e.g. outlier detection, or clustering analysis) and iv) actionable recommendations.

### Ideas:
1. Create Lyft-rate object for easy-usage of lyft rate on card
2. Seems like the DLV can be tackled in orders of increasing complexity: i) take the average on_boarded time, until the very last ride available and average over all drivers (Naive soln), ii) CV linear regression based on features, define Y as onboarded_time - last_ride, iii) more complex CV regressions (e.g. RF)
3. Clustering seems straightforward. Need to identify the most profitable driver segment for lyft, and attributes available in dataset corresponding to them.

# DLV Conceptual Ideas:
1. ride type (mileage vs time): [X]
    * compute % (i.e. ratio) of total preservice/base fare of mileage, and time
2. onboard_time: [x]
    * compute time between onboard, and first ride
    * compute time between first ride and last ride

## Not Done Yet:
1. Longer the driver is w/ a comp -> generally more value
    - weight longer time they're w/ company
    - assuming riders don't drop out
        -> figure out if "drivers even dropped out". How to define this.
    -> multiple functions of time, for an "estimated" value. (maybe define 2-5), for us to go off of
    -> duration between onboard and first ride 
    -> duration between first ride and last ride
    
2. The more incompleted rides, the worse
    -> requested, but canceled
    -> accepted, but canceled
    -> arrived at, but canceled
    -> picked up at, but canceled
    
3. The more completed rides "in general, the better
    -> all 5 from same ride id, -> mark rides completed, or not column

4. Type of ride - timely, versus long-distance
    -> ratio of distance to cost vs duration to the cost
    
5. Time of day - earlymorning, morning, afternoon, evening, late-evening
    -> 
    
6. Time of week - weekday, weekend, friday night?
    ->

7. Week of month? 1-4
    
8. PrimeTime dependence?
    - driver's frequency of primetime per week, per month
    - driver's amount of primetimes overall / the total amount of time spent

9. Adding data columns:
    - number max fares per driver
    - driver's correlation/dependence on distance = average of the "type of ride"
    - 

# Categorizations of Drivers
1. Full time / parttime / other?

2. high frequency, or consistent?
    - driving all the time, or driving only on certain days of the week


In [12]:
# Necessary libs
import os
import numpy as np
import pandas as pd
import scipy
import sklearn

# Plotting
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import dabest

# custom libs
import sys
sys.path.append("../")
from src.data.rider import Fare, Ride

# Import magic commands for jupyter notebook 
# - autoreloading a module
# - profiling functions for memory usage and scripts
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## 1. Load in Data - DataFrame

In [15]:
datadir = "../data/raw/"
rideid_fpath = os.path.join(datadir, "ride_ids.csv")
driveid_fpath = os.path.join(datadir, "driver_ids.csv")
ridetime_fpath = os.path.join(datadir, "ride_timestamps.csv")
                              
ride_df = pd.read_csv(rideid_fpath)
drive_df = pd.read_csv(driveid_fpath)
time_df = pd.read_csv(ridetime_fpath)

print(os.listdir(datadir))

['.gitkeep', 'ride_timestamps.csv', 'driver_ids.csv', 'ride_ids.csv']


In [16]:
display(ride_df.head())
display(drive_df.head())
display(time_df.head())

print("The size of the dataframes are: ", ride_df.shape, drive_df.shape, time_df.shape)

,driver_id,ride_id,ride_distance,ride_duration,ride_prime_time
0,002be0ffdc997bd5c50703158b7c2491,006d61cf7446e682f7bc50b0f8a5bea5,1811,327,50
1,002be0ffdc997bd5c50703158b7c2491,01b522c5c3a756fbdb12e95e87507eda,3362,809,0
2,002be0ffdc997bd5c50703158b7c2491,029227c4c2971ce69ff2274dc798ef43,3282,572,0
3,002be0ffdc997bd5c50703158b7c2491,034e861343a63ac3c18a9ceb1ce0ac69,65283,3338,25
4,002be0ffdc997bd5c50703158b7c2491,034f2e614a2f9fc7f1c2f77647d1b981,4115,823,100


,driver_id,driver_onboard_date
0,002be0ffdc997bd5c50703158b7c2491,2016-03-29 00:00:00
1,007f0389f9c7b03ef97098422f902e62,2016-03-29 00:00:00
2,011e5c5dfc5c2c92501b8b24d47509bc,2016-04-05 00:00:00
3,0152a2f305e71d26cc964f8d4411add9,2016-04-23 00:00:00
4,01674381af7edd264113d4e6ed55ecda,2016-04-29 00:00:00


,ride_id,event,timestamp
0,00003037a262d9ee40e61b5c0718f7f0,requested_at,2016-06-13 09:39:19
1,00003037a262d9ee40e61b5c0718f7f0,accepted_at,2016-06-13 09:39:51
2,00003037a262d9ee40e61b5c0718f7f0,arrived_at,2016-06-13 09:44:31
3,00003037a262d9ee40e61b5c0718f7f0,picked_up_at,2016-06-13 09:44:33
4,00003037a262d9ee40e61b5c0718f7f0,dropped_off_at,2016-06-13 10:03:05


The size of the dataframes are:  (193502, 5) (937, 2) (970405, 3)


## 1a) Cleanup and Distribution Analysis

- Looks like all the ride_ids and driver_ids are in these abhorent numbers, so we can create a dictionary instead to clean them up.
- Need to convert meters into miles to compute the fare per distance
- Need to convert the duration from seconds into minutes
- Looks like dates are all in datetime format, so also add column in unix time to get a sense of total distribution. 
    
    pd.to_datetime(['2019-01-15 13:30:00']).astype(int) / 10**9
    
- summarize total ranges of riders, drivers, distances, durations, primetime, time range analysis 

In [17]:
unique_driver_ids = {x: f"driver_{i}" for i, x in enumerate(drive_df.driver_id.unique())}
unique_ride_ids = {x: f"ride_{i}" for i, x in enumerate(ride_df.ride_id.unique())}

print(len(unique_driver_ids), " unique riders with data for their onboarding time.")
print(len(unique_ride_ids))

937  unique riders with data for their onboarding time.
193502


In [18]:
drive_df["driver_id"] = drive_df["driver_id"].map(unique_driver_ids)
ride_df["ride_id"] = ride_df["ride_id"].map(unique_ride_ids)
ride_df["driver_id"] = ride_df["driver_id"].map(unique_driver_ids)
time_df["ride_id"] = time_df["ride_id"].map(unique_ride_ids)

display(ride_df.head(2))
display(drive_df.head(2))
display(time_df.head(2))

,driver_id,ride_id,ride_distance,ride_duration,ride_prime_time
0,driver_0,ride_0,1811,327,50
1,driver_0,ride_1,3362,809,0


,driver_id,driver_onboard_date
0,driver_0,2016-03-29 00:00:00
1,driver_1,2016-03-29 00:00:00


,ride_id,event,timestamp
0,ride_167241,requested_at,2016-06-13 09:39:19
1,ride_167241,accepted_at,2016-06-13 09:39:51


In [19]:
# convert meters to miles
def getMiles(i):
     return i*0.000621371192

def getMeters(i):
     return i*1609.344
    
ride_df['ride_distance_miles']=ride_df['ride_distance'].apply(getMiles)

In [20]:
# convert durations from seconds -> minutes, keep as float
def getMinutes(i):
    return i / 60.0

ride_df['ride_duration_mins']=ride_df['ride_duration'].apply(getMinutes)

In [21]:
# convert to unix time and add
drive_df["driver_onboard_date_unix"] = pd.to_datetime(drive_df["driver_onboard_date"]).astype(int) / 10**9
time_df["timestamp_unix"] = pd.to_datetime(time_df["timestamp"]).astype(int) / 10**9

display(drive_df.head(2))
display(time_df.head(2))

,driver_id,driver_onboard_date,driver_onboard_date_unix
0,driver_0,2016-03-29 00:00:00,1.459210e+09
1,driver_1,2016-03-29 00:00:00,1.459210e+09


,ride_id,event,timestamp,timestamp_unix
0,ride_167241,requested_at,2016-06-13 09:39:19,1.465811e+09
1,ride_167241,accepted_at,2016-06-13 09:39:51,1.465811e+09


In [22]:
ride_df['ride_prime_time'] = ride_df['ride_prime_time'] / 100

In [32]:
%%time
# takes a bit of time to run, ideally finish running everything and save
fare_cols = [
    'ride_prime_time',
    'ride_distance_miles',
    'ride_duration_mins',
]

def compute_fare(vec):
    vec = vec.tolist()
    ride = Ride(vec[0], vec[1], vec[2])
    return pd.Series([ride.fare, np.round(ride.mileage_percentage, 4), np.round(ride.time_percentage, 4)])

# pipeline of additions
ride_df[["fare_total", "mile_perc", "time_perc"]] = ride_df[fare_cols].apply(compute_fare, axis=1)

display(ride_df.head())

,driver_id,ride_id,ride_distance,ride_duration,ride_prime_time,ride_distance_miles,ride_duration_mins,fare_total,mile_perc,time_perc
0,driver_0,ride_0,1811,327,0.50,1.125303,5.450000,12.764305,0.8228587526526373,0.17714124734736278
1,driver_0,ride_1,3362,809,0.00,2.089050,13.483333,27.062243,0.7144672207110176,0.2855327792889823
2,driver_0,ride_2,3282,572,0.00,2.039340,9.533333,20.589164,0.7193547725851649,0.2806452274148351
3,driver_0,ride_3,65283,3338,0.25,40.564976,55.633333,400.000000,0.11415191274751688,0.8858480872524831
4,driver_0,ride_4,4115,823,1.00,2.556942,13.716667,45.876861,0.803485775582325,0.1965142244176749


CPU times: user 1min 56s, sys: 2.01 s, total: 1min 58s
Wall time: 2min 4s


# Merge Dataframes

Some data is in other dataframes pertaining to the driver. Such as onboard date, so we will merge that in w/ the ride_df.

Looks like onboard date is 1-1, so let's just create multiple dictionaries to store the corresponding data we are interested in:

- month
- day of week
- year

In [14]:
display(ride_df.head(2))
display(drive_df.head(2))
display(time_df.head(2))

,driver_id,ride_id,ride_distance,ride_duration,ride_prime_time,ride_distance_miles,ride_duration_mins,fare_total
0,driver_0,ride_0,1811,327,0.5,1.125303,5.450000,5.169000
1,driver_0,ride_1,3362,809,0.0,2.089050,13.483333,10.122667


,driver_id,driver_onboard_date,driver_onboard_date_unix
0,driver_0,2016-03-29,1.459210e+09
1,driver_1,2016-03-29,1.459210e+09


,ride_id,event,timestamp,timestamp_unix
0,ride_167241,requested_at,2016-06-13 09:39:19,1.465811e+09
1,ride_167241,accepted_at,2016-06-13 09:39:51,1.465811e+09


In [36]:
time_df['timestamp'] = pd.to_datetime(time_df['timestamp'])

In [33]:
drive_df['driver_onboard_date'] = pd.to_datetime(drive_df['driver_onboard_date'])
driver_id = drive_df['driver_id']
onboard_month = {_id: x for _id, x in zip(driver_id, drive_df['driver_onboard_date'].dt.month)}
onboard_day = {_id: x for _id, x in zip(driver_id, drive_df['driver_onboard_date'].dt.day)}
onboard_year = {_id: x for _id, x in zip(driver_id, drive_df['driver_onboard_date'].dt.year)}

In [37]:
driver_df = pd.merge(ride_df, drive_df, on="driver_id")

print(driver_df.shape)
display(driver_df.head())

(185891, 12)


,driver_id,ride_id,ride_distance,ride_duration,ride_prime_time,ride_distance_miles,ride_duration_mins,fare_total,mile_perc,time_perc,driver_onboard_date,driver_onboard_date_unix
0,driver_0,ride_0,1811,327,0.50,1.125303,5.450000,12.764305,0.8228587526526373,0.17714124734736278,2016-03-29,1.459210e+09
1,driver_0,ride_1,3362,809,0.00,2.089050,13.483333,27.062243,0.7144672207110176,0.2855327792889823,2016-03-29,1.459210e+09
2,driver_0,ride_2,3282,572,0.00,2.039340,9.533333,20.589164,0.7193547725851649,0.2806452274148351,2016-03-29,1.459210e+09
3,driver_0,ride_3,65283,3338,0.25,40.564976,55.633333,400.000000,0.11415191274751688,0.8858480872524831,2016-03-29,1.459210e+09
4,driver_0,ride_4,4115,823,1.00,2.556942,13.716667,45.876861,0.803485775582325,0.1965142244176749,2016-03-29,1.459210e+09


In [38]:
driver_df = pd.merge(driver_df, time_df, on="ride_id")

print(driver_df.shape)
display(driver_df.head(2))

(921045, 15)


,driver_id,ride_id,ride_distance,ride_duration,ride_prime_time,ride_distance_miles,ride_duration_mins,fare_total,mile_perc,time_perc,driver_onboard_date,driver_onboard_date_unix,event,timestamp,timestamp_unix
0,driver_0,ride_0,1811,327,0.5,1.125303,5.45,12.764305,0.8228587526526373,0.17714124734736278,2016-03-29,1.459210e+09,requested_at,2016-04-23 02:13:50,1.461378e+09
1,driver_0,ride_0,1811,327,0.5,1.125303,5.45,12.764305,0.8228587526526373,0.17714124734736278,2016-03-29,1.459210e+09,accepted_at,2016-04-23 02:14:15,1.461378e+09
2,driver_0,ride_0,1811,327,0.5,1.125303,5.45,12.764305,0.8228587526526373,0.17714124734736278,2016-03-29,1.459210e+09,arrived_at,2016-04-23 02:16:36,1.461378e+09
3,driver_0,ride_0,1811,327,0.5,1.125303,5.45,12.764305,0.8228587526526373,0.17714124734736278,2016-03-29,1.459210e+09,picked_up_at,2016-04-23 02:16:40,1.461378e+09
4,driver_0,ride_0,1811,327,0.5,1.125303,5.45,12.764305,0.8228587526526373,0.17714124734736278,2016-03-29,1.459210e+09,dropped_off_at,2016-04-23 02:22:07,1.461378e+09


In [42]:
%%time
# add onboarding time and time between first and last rides
def compute_onboardtime_toride(df):
    drivers = df['driver_id'].unique()
    onboard_event_dates = df.loc[df['driver_id'].isin(drivers)][['driver_onboard_date', 'timestamp']]
    return onboard_event_dates["timestamp"] - onboard_event_dates["driver_onboard_date"]

# pipeline of additions
driver_df['time_since_onboard'] = driver_df.pipe(compute_onboardtime_toride)

display(driver_df.head(2))

,driver_id,ride_id,ride_distance,ride_duration,ride_prime_time,ride_distance_miles,ride_duration_mins,fare_total,mile_perc,time_perc,driver_onboard_date,driver_onboard_date_unix,event,timestamp,timestamp_unix,time_since_onboard
0,driver_0,ride_0,1811,327,0.5,1.125303,5.45,12.764305,0.8228587526526373,0.17714124734736278,2016-03-29,1.459210e+09,requested_at,2016-04-23 02:13:50,1.461378e+09,-26 days +21:46:10
1,driver_0,ride_0,1811,327,0.5,1.125303,5.45,12.764305,0.8228587526526373,0.17714124734736278,2016-03-29,1.459210e+09,accepted_at,2016-04-23 02:14:15,1.461378e+09,-26 days +21:45:45
2,driver_0,ride_0,1811,327,0.5,1.125303,5.45,12.764305,0.8228587526526373,0.17714124734736278,2016-03-29,1.459210e+09,arrived_at,2016-04-23 02:16:36,1.461378e+09,-26 days +21:43:24
3,driver_0,ride_0,1811,327,0.5,1.125303,5.45,12.764305,0.8228587526526373,0.17714124734736278,2016-03-29,1.459210e+09,picked_up_at,2016-04-23 02:16:40,1.461378e+09,-26 days +21:43:20
4,driver_0,ride_0,1811,327,0.5,1.125303,5.45,12.764305,0.8228587526526373,0.17714124734736278,2016-03-29,1.459210e+09,dropped_off_at,2016-04-23 02:22:07,1.461378e+09,-26 days +21:37:53


CPU times: user 343 ms, sys: 424 ms, total: 767 ms
Wall time: 835 ms


In [47]:
%%time
# add onboarding time and time between first and last rides
def compute_onboardtime_tofirstride(df):
    drivers = df['driver_id'].unique()
    answers = {}
    for _driver in drivers:
        first_event = df.loc[df['driver_id'] == _driver][['timestamp']].min()
        onboard_date = df.loc[df['driver_id'] == _driver][['driver_onboard_date']]
        answers[_driver] = first_event - onboard_date
    df['time_to_firstride'] = df["driver_id"].map(answers)
    return df

# pipeline of additions
driver_df = driver_df.pipe(compute_onboardtime_tofirstride)

display(driver_df.head(2))

,driver_id,ride_id,ride_distance,ride_duration,ride_prime_time,ride_distance_miles,ride_duration_mins,fare_total,mile_perc,time_perc,driver_onboard_date,driver_onboard_date_unix,event,timestamp,timestamp_unix,time_since_onboard,time_to_firstride
0,driver_0,ride_0,1811,327,0.5,1.125303,5.45,12.764305,0.8228587526526373,0.17714124734736278,2016-03-29,1.459210e+09,requested_at,2016-04-23 02:13:50,1.461378e+09,-26 days +21:46:10,driver_onboard_date timestamp 0 ...
1,driver_0,ride_0,1811,327,0.5,1.125303,5.45,12.764305,0.8228587526526373,0.17714124734736278,2016-03-29,1.459210e+09,accepted_at,2016-04-23 02:14:15,1.461378e+09,-26 days +21:45:45,driver_onboard_date timestamp 0 ...


CPU times: user 2min 39s, sys: 1.02 s, total: 2min 40s
Wall time: 2min 42s


# 2) Save intermediate dataframes into data

- Save all dataframes into ../data/interim/


In [48]:
fpath = "../data/interim/driver_v1.csv"

In [49]:
driver_df.to_csv(fpath)

KeyboardInterrupt: 

# Categorizations of Drivers
1. Full time / parttime / other?

2. high frequency, or consistent?
    - driving all the time, or driving only on certain days of the week


# 1b) Visualize exploration results